# Proyecto 2

## Aprendizaje no supervisado sobre perfiles de una red de contactos

Información sobre perfiles de la web de contactos okcupid.com (web de citas)

Determinar qué perfiles de la red social son compatibles entre sí

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition  import LatentDirichletAllocation
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer

In [2]:
profiles = pd.read_csv('okcupid_profiles.csv', header='infer')
profiles.shape

(59946, 31)

In [3]:
profiles.tail(50)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
59896,26,available,f,straight,curvy,mostly vegetarian,often,sometimes,graduated from college/university,white,...,i'm recently married and my husband cathartid ...,i am an environmental educator but i'm current...,breaking stuff. sigh....,NaN,i love books! i just finished a sand county al...,music that gives me chills the sun on my bare...,NaN,"if i have been drinking, i'm probably telling ...","sometimes, when i'm alone in the bathroom, i l...",you like what you've read above and on cathart...
59897,30,single,f,straight,full figured,NaN,socially,never,working on two-year college,"black, white",...,i'm in a really good place personally and prof...,living it one day at a time...,NaN,my unique appearance..,NaN,family friends laughter my dog my iphone $$$- ...,NaN,usually out with friends,i was once at a reasturant and laughed so hard...,NaN
59898,34,single,f,straight,thin,mostly anything,socially,NaN,working on space camp,"hispanic / latin, white",...,"i'm an honest, hard working single mom looking...",i'll let you know when i figure it out.,cooking. organizing the dishwasher. dancing al...,maybe i'll start asking people that.,"books: twilight, john grisham, vc andrews. mov...",1. kids 2. family 3. water (the ocean) 4. asia...,how to give my kids a better life. what i want...,"exhausted from work, or out looking for fun.",that i may be wrong sometimes (but i won't adm...,serious inquiries only please. please have you...
59899,23,single,f,straight,thin,strictly other,socially,never,NaN,white,...,i come off pretty rough to people. mainly beca...,i'm currently in the prologue of my life. i kn...,being extremely clumsy. saying the wrong thing...,"are you russian? actually i'm ukrainian, but i...",NaN,there's definitely more than six things i coul...,"the future.. pretty generic answer, but it's t...",out!,nothing is ever too private.,i'm interested in meeting all sorts of people....
59900,27,single,f,straight,fit,strictly anything,socially,NaN,graduated from college/university,white,...,"oh boy, here we go....from portland, or born a...",i am a sales engineer for a tech company in su...,"not taking myself too seriously, being sarcast...",my smile. and my sass. watch out...,"major food enthusiast (if i say ""foodie"" does ...",the things you own end up owning you...,travel travel travel. i like it. a lot. been a...,oot and aboot,i can't bring myself to let the microwave soun...,"you are a non-creeper...tall order, i know."
59901,22,available,f,bisexual,fit,NaN,socially,sometimes,working on college/university,NaN,...,hella hella hella hella hella hella hella hell...,"i'm an aspiring musician, pharmacist or scient...",driving. and referencing movies.,my red hair. why do people ask me what color m...,"i love reading, but i have never finished a bo...","my journal, my glasses, my car, my cat, food, ...","if animals could talk, what accent would they ...",doing whatever i feel like doing that night,i have a stuffed animal bunny that i've had si...,you: 1) can catch on to my sarcasm 2) don't sp...
59902,26,single,f,straight,curvy,mostly other,rarely,never,graduated from space camp,"native american, hispanic / latin, white",...,"""to love things as if they are people is an il...",enjoying every moment of it...,crafty stuff.. painting nails.. making sweet t...,i think right now its probably my hair.. its p...,"books: on the road, mexico city blues, big sur...",water-love-laughter-icecream-potion#9-nail polish,trying not to think too much..,hanging out with friends.. or relaxing around ...,i recently had to stop drinking vodka.. i thin...,if you'd like to offer me anything other than ...
59903,35,single,f,straight,fit,mostly anything,socially,never,graduated from college/university,white,...,NaN,still working in the restaurant business but i...,being myself,my laugh,blood meridian the master and

In [4]:
#Variables continuas
profiles.describe()

,age,height,income
count,59946.000000,59943.000000,59946.000000
mean,32.340290,68.295281,20033.222534
std,9.452779,3.994803,97346.192104
min,18.000000,1.000000,-1.000000
25%,26.000000,66.000000,-1.000000
50%,30.000000,68.000000,-1.000000
75%,37.000000,71.000000,-1.000000
max,110.000000,95.000000,1000000.000000


In [5]:
#profiles.groupby(by='status').count()

#available+single --> preguntar: es distinto, dejarlas como estan
#married
#seeing someone
#unknown

In [6]:
#profiles.groupby(by='sex').count()

In [7]:
#profiles.groupby(by='orientation').count()

#Bisexual 1 1
#One hot encoding solo con 2 columnas

In [8]:
#profiles.groupby(by='body_type').count()

#Drop --> preguntar

In [9]:
#profiles.groupby(by='diet').count()

In [10]:
#profiles.groupby(by='drinks').count()

In [11]:
#profiles.groupby(by='drugs').count()

#Sí/No

In [12]:
#profiles.groupby(by='education').count()

#Preguntar si es relevante y si es repetitiva con el job

In [13]:
#profiles.groupby(by='ethnicity').count()


In [14]:
#profiles.groupby(by='job').count()

In [15]:
#profiles.groupby(by='last_online').count()

In [16]:
#profiles.groupby(by='location').count()

In [17]:
#profiles.groupby(by='offspring').count()

In [18]:
#profiles.groupby(by='pets').count()

In [19]:
#profiles.groupby(by='religion').count()

In [20]:
#profiles.groupby(by='sign').count()

In [21]:
#profiles.groupby(by='smokes').count()

In [22]:
#profiles.groupby(by='speaks').count()

## Exploración de los valores nulos

In [23]:
profiles.isna().sum()

age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
dtype: int64

## Fusión de las nueve columnas 'essay0'...'essay9' a 'essay'

In [24]:
#profiles.essay0.isna().sum()
#profiles.essay0[45497]
#profiles.essay0[45497] = ' '
#profiles.essay0[45497]

In [25]:
profiles.head()

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,about me: i would love to think that i was so...,currently working as an international agent fo...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,i am a chef: this is what that means. 1. i am ...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,NaN,NaN,i am very open and will share just about anyth...,NaN
2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,NaN,...,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement conversation creation contemplation t...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ..."
3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,white,...,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,"bataille, celine, beckett. . . lynch, jarmusch...",NaN,cats and german philosophy,NaN,NaN,you feel so inclined.
4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",...,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at: http://bagsbrown....,i smile a lot and my inquisitive nature,"music: bands, rappers, musicians at the moment...",NaN,NaN,NaN,NaN,NaN


### Sustitución de valores nan por string vacío

In [28]:
# Extraer los indices de los valores nan en una lista y sustituirlos por ' '
for j in np.arange(10):
    indices = profiles.index[profiles['essay'+str(j)].isna()].tolist()  #saca una lista con los indices de los valores que estan a nan
    # profiles.index[profiles.essay0.isna()].value_counts().sum()  para saber cuantos son
    for ind in indices:
        profiles.loc['essay'+str(j)][ind] = ' '

In [29]:
# Comprobar que no hay valores nan desde essay0 ... hasta essay9
profiles.isna().sum()

age                0
status             0
sex                0
orientation        0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
pets           19921
religion       20226
sign           11056
smokes          5512
speaks            50
essay0             0
essay1             0
essay2             0
essay3             0
essay4             0
essay5             0
essay6             0
essay7             0
essay8             0
essay9             0
dtype: int64

### Nueva columna all_essays

In [30]:
#Unificacion de todos los essays a una sola columna
#columns = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
essays = []

essays = profiles.essay0 + ' ' + profiles.essay1 + ' ' + profiles.essay2 + ' ' + profiles.essay3 + ' ' + profiles.essay4 + ' ' + profiles.essay5 + ' ' + profiles.essay6 + ' ' + profiles.essay7 + ' ' + profiles.essay8 + ' ' + profiles.essay9

In [31]:
#comprobamos que no hay ningun valor nan en essays
essays.isna().sum()

0

In [32]:
len(essays)

59946

In [33]:
#Nueva columna en pandas con la fusion
profiles['all_essays'] = essays

In [14]:
#for i in range(len(profiles)):
    #for j in columns:
        #lista.append(profiles[j][i])
    #essays.append(lista)

In [15]:
#profiles['essay0'][0]

In [34]:
profiles.tail(2)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,all_essays
59944,27,single,m,straight,athletic,mostly anything,socially,often,working on college/university,"asian, black",...,"following my dreams... ""you got a dream... you...",listening,it used to be the hair until i mowed it off bu...,where to begin musically: right now i listen t...,"music, family, friends, a basketball, hoop, so...",what can i do to make someone chuckle....,what i would do on any other day. everydays a ...,i like walking around in other people's house ...,you are interested and interesting...,"""all i have in this world are my balls and my ..."
59945,39,single,m,gay,average,NaN,socially,NaN,graduated from masters program,white,...,i work with elderly people (psychotherapy and ...,i'm a great bullshitter. i don't know what it ...,"either that i am funny/sarcastic, or that i am...","i just read the help by kathryn stockett, sooo...",1. family & friends & other humans - interacti...,"sex, myself, other people, how amazing everyth...","out at happy hour with my friends, running int...",i wish i could cry like holly hunter in broadc...,"if you have a back-bone, an opinion, a sense o...","is it odd that having a little ""enemy"" status ..."


## Preparación del texto

In [35]:
essays

0        about me:  i would love to think that i was so...
1        i am a chef: this is what that means. 1. i am ...
2        i'm not ashamed of much, but writing public te...
3        i work in a library and go to school. . . read...
4        hey how's it going? currently vague on the pro...
                               ...                        
59941    vibrant, expressive, caring optimist. i love b...
59942    i'm nick. i never know what to write about mys...
59943    hello! i enjoy traveling, watching movies, and...
59944    "all i have in this world are my balls and my ...
59945    is it odd that having a little "enemy" status ...
Length: 59946, dtype: object

In [37]:
#Fusion de todas las filas de essays para procesar el texto entero y sacar los n topics que mas se repiten
#mix = ''
#for i in range(5000):
#    mix += ' ' + essays[i]

In [38]:
#mix
#len(essays)
#essays[0]

In [157]:
#Customizing stopwords
new_words = ['still','take','trying','great','go','little','live','good','big','years','love','right','back','mine','thing','things',
             'better','anything','really','able','half','made','way','make','someone','everything','kind','see','type','types','new',
             'more','much','lot','though','unless','probably','usually','looking','also','something','know','think','like','enjoy',
             'well','time','making','always','try','one','would','around','many','making','going','want','trying','open','long',
             'pretty','say','sometimes','getting','best','stuff','person','day','favorite','feel','old','smile','guy','life','living',
             'afluster', 'dfg', 'eleifend', 'emfoi', 'ewoe', 'faucibus', 'fgd', 'frakking', 'fsd', 'fsdf', 'ghtr', 'hilda', 'imm',
             'maecenas', 'mafoiawm', 'malesuada', 'moewaaoicmw', 'moiwa', 'mweiof', 'nn', 'oma', 'pharetra', 'praline', 'rg', 'rth',
             'saah', 'sdf', 'sfd', 'thous', 'torolf', 'utm', 'wahh', 'woi', 'wordsi', 'yuval','aoicmw', 'bhakti', 'bluetoonist', 'citation',
             'curabitur', 'dfgd', 'epr', 'gosto', 'gsdsd', 'hallie', 'inhabitable', 'lobortis', 'lorelai', 'lorry', 'lovebot', 'luctus',
             'mdmj', 'moewa', 'ner', 'pandit', 'prema', 'proident', 'realmente', 'rgd', 'sfsd', 'sgi', 'slats', 'sugalumps',
             'superintendent', 'teehehe', 'tramps', 'ustad', 'varius', 'vulputate', 'yew', 'annun', 'artsier', 'autobus',
             'congue', 'enfjs', 'eplariza', 'exercitation', 'fru', 'gatorar', 'gazelle', 'godhead', 'incididunt', 'infps',
             'jrpg', 'kenner', 'kielbasa', 'lipps', 'meep', 'messi', 'motherflippin', 'nfnc', 'nostrud', 'ralston', 'ratkins',
             'roiphe', 'ruffin', 'rutrum', 'saraswati', 'sifu', 'stefisbuff', 'strongbow', 'suscipit', 'tili', 'unst', 'walla', 'aliqua',
             'aliquip', 'andouille', 'apna', 'ashzra', 'auch', 'congruence', 'cupidatat', 'demisexual', 'dolore', 'ecoutez', 'elly', 'frnds',
             'fuckups', 'fuk', 'kladdkaka', 'klove', 'laboris', 'lov', 'lupus', 'melia', 'nao', 'omnicircus', 'paciocco', 'quidam', 'rere',
             'setters', 'sharpens', 'singe', 'sint', 'teksto', 'thaat', 'tribunal', 'waaahh', 'wawah', 'aglets', 'akashic', 'belladonna', 'boingy',
             'breatharian', 'buk', 'cafepress', 'cyclenator', 'ecopsychology', 'esse', 'excetera', 'fnordloki', 'fortran', 'gado', 'hoon',
             'horsey', 'krylon', 'lawton', 'noc', 'occitan', 'officia', 'orale', 'pawns', 'pwc', 'ql', 'reactivity', 'rutgers', 'sexanddesign',
             'slocks', 'thoe', 'toga', 'tudi', 'voluptate', 'wedged', 'zander','adipisicing', 'alota', 'arnall', 'arrr', 'babygirl',
             'bjrn', 'chanchito', 'consonant', 'dob', 'dunh', 'duval', 'eyeware', 'ftapon', 'gallbladder', 'guiltied', 'gust', 'haulin',
             'jetzt', 'kma', 'lili', 'matchy', 'melinda', 'moocow', 'muzix', 'mysore', 'oshun', 'porttitor', 'prendre', 'rnr', 'saige',
             'sattvic', 'tabor', 'tartan', 'teaspoons', 'unemotional','aanq', 'aku', 'alg', 'allister', 'aoi', 'debrief', 'estar', 'felling',
             'fw', 'guyana', 'hilbert', 'hornier', 'humin', 'jjc', 'joyfulparadox', 'larm', 'layne', 'livros', 'maj', 'mii', 'muziq', 'newsstand',
             'nullalux', 'oooooooooaaaaaaawwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
             'peices', 'pitted', 'quelque', 'resently', 'rly', 'sleeeeeping', 'supplementation', 'topher', 'tristissima', 'viel', 'virgule', 'people','world', 'fun', 'work', 'working']

stopwords_customized = stopwords.words('english') + new_words + list(ENGLISH_STOP_WORDS)


In [55]:
def preprocess_text(text):
#for i in range(len(text)):
    #corpus = []
    review = re.sub(r'\W', ' ', text) #sentences[i]     # Sustituye los caracteres no alfanuméricos por espacio
    review = review.lower()                             # Conversion a minusculas
    review = re.sub(r'^br$', ' ', review)               # Los que empiecen por 'b' y terminen por 'r' se sustituyen por un espacio en blanco
    review = re.sub(r'\s+br\s+',' ',review)             # El patron (espacio en blanco o más, br, espacio en blanco o mas) se sustituye por un espacio en blanco
    review = re.sub(r'\s+[a-z]\s+', ' ',review)         # El patron (espacio en blanco o más, cualquier letra del alfabeto, espacio en blanco o más) se sustituye por un espacio en blanco
    review = re.sub(r'^b\s+', '', review)               # Comienzo de la cadena con 'b' seguido de uno o mas espacios en blanco lo elimina
    review = re.sub(r'\s+', ' ', review)                # Eliminar los espacios en blanco sobrantes incluidos en los pasos anteriores.
    review = re.sub('[^a-zA-Z]+', ' ', review)
    #corpus.append(review)r'\s+'
    return review

def tokenize(text):
    sentences = nltk.sent_tokenize(text)
    return sentences


In [95]:
#corpus_tokenizado = tokenize(mix)

In [96]:
#len(corpus_tokenizado)

117064

In [67]:
essays[0]

[' ']

In [78]:
preprocess_text('i would consider my self very ambitious out going person who really just whants the best out of life m not scared of much im willing to try just about anything at least once m really in to being active and eating healthy look at life like it short and better enjoy it while got it one love am entrapinor have lots of endevors im curently working on and many life goals reach for the stars on daily right now m in the process of starting clothing company have lots of other things that are in the workings aswell iv decided am going to make my life what whant it to be and there will be no other way trying to not listen to hatters would say am really good at alot of things m good at solving problems when the arise m good at skating snow boarding dirt biking racing cars loving life looking at the positive hoping for the best growing things the list goes on is that m pretty eazy going and that go way out of my way for those care about movies pulp fiction one flew over the cocos nest friday hang over limitless kill bill too many more to list music bass nectar dead mouse mac dre biggie bob marley barrington levy andre nickatina antique beats lil wayne mos def living legends hieroglyphics wu tang drake uk dub step and seriously so much more shows breaking bad seinfeld southpark simpsons dexter family guy road runner some csi weed wars fear factor always sunny in philidelphia there lots other good shows aswell books fight club think and grow rich lord of the rings rich dad poor dad willie wonka and the chocolate factory lot of others foods sushi pizza thai mexican indian chinese healthy sustainable organic cooking allsorts egg breakfast of all types ice cream fruit there to much good food to list water art music warm clothes in winter love the sun beautiful women like ur self am ready for anything some times like to chill and take it easy sometimes like to get faded and rage just depends on whats going on if like to have fun if whant to get crazy if need some one to remind how good life really is ')

'i would consider my self very ambitious out going person who really just whants the best out of life not scared of much im willing to try just about anything at least once really in to being active and eating healthy look at life like it short and better enjoy it while got it one love am entrapinor have lots of endevors im curently working on and many life goals reach for the stars on daily right now in the process of starting clothing company have lots of other things that are in the workings aswell iv decided am going to make my life what whant it to be and there will be no other way trying to not listen to hatters would say am really good at alot of things good at solving problems when the arise good at skating snow boarding dirt biking racing cars loving life looking at the positive hoping for the best growing things the list goes on is that pretty eazy going and that go way out of my way for those care about movies pulp fiction one flew over the cocos nest friday hang over limitl

In [57]:
fusion = []
for i in range(len(essays)):
    fusion.append(preprocess_text(essays[i]))

In [58]:
len(fusion)

59946

In [106]:
fusion

['about me would love to think that was some some kind of intellectual either the dumbest smart guy or the smartest dumb guy can say can tell the difference love to talk about ideas and concepts forge odd metaphors instead of reciting cliches like the simularities between friend of mine house and an underwater salt mine my favorite word is salt by the way weird choice know to me most things in life are better as metaphors seek to make myself little better everyday in some productively lazy way got tired of tying my shoes considered hiring five year old but would probably have to tie both of our shoes decided to only wear leather shoes dress shoes about you you love to have really serious really deep conversations about really silly stuff you have to be willing to snap me out of light hearted rant with kiss you don have to be funny but you have to be able to make me laugh you should be able to bend spoons with your mind and telepathically make me smile while am still at work you should 

In [46]:
corpus = preprocess_text(mix)

In [49]:
corpus

['y']

In [47]:
len(corpus)

1

In [161]:
#Convert a collection of text documents to a matrix of token counts
vectorizer = CountVectorizer(max_features = 20, min_df = 0.25, max_df = 0.85, stop_words = stopwords_customized ) #Convert a collection of text documents to a matrix of token counts
X = vectorizer.fit_transform(fusion).toarray()

In [162]:
X.shape

(59946, 10)

In [163]:
vectorizer.vocabulary_

{'reading': 9,
 'books': 0,
 'movies': 6,
 'music': 7,
 'food': 2,
 'friends': 3,
 'read': 8,
 'movie': 5,
 'family': 1,
 'home': 4}

In [164]:
# Mostrar las características.
features = vectorizer.get_feature_names()
print(features)

['books', 'family', 'food', 'friends', 'home', 'movie', 'movies', 'music', 'read', 'reading']


In [165]:
######LDA
# Obtener los temas.
n_topics = 4
lda = LatentDirichletAllocation(n_components=n_topics, random_state=123)
my_docs_topic = lda.fit_transform(X)  

In [166]:
my_docs_topic

array([[0.03689712, 0.32303481, 0.60387851, 0.03618957],
       [0.12777698, 0.12540162, 0.620254  , 0.1265674 ],
       [0.50949537, 0.27760066, 0.02624686, 0.18665711],
       ...,
       [0.48125781, 0.02014014, 0.34803398, 0.15056807],
       [0.61785289, 0.31298927, 0.0367009 , 0.03245695],
       [0.22491138, 0.55071749, 0.1153142 , 0.10905693]])

In [167]:
topic_composition = lda.components_

In [170]:
n_top = 4
for i in range(n_topics):
    topic_features = [features[idx] for idx in np.argsort(-topic_composition[i,:])]   # argsort() muestra el índice ordenado.
    topic_features_top = topic_features[0:n_top]
    if i == 0:
        topic_matrix = [topic_features_top]                    # list의 list 만들 준비!
    else:
        topic_matrix.append(topic_features_top) 

In [171]:
# Muestra las características principales de cada tema.
topic_matrix

[['friends', 'family', 'food', 'music'],
 ['read', 'reading', 'books', 'movies'],
 ['music', 'food', 'movies', 'books'],
 ['home', 'movie', 'friends', 'family']]

In [172]:
# En vista de las características principales, podemos nombrar los temas.
topic_names = ['Social','Book','Music','Movie']

In [174]:
# El tema más probable viene dado directamente por la salida de LDA.
n_docs = len(essays)
labels = []
for i in range(n_docs):
    topic_pick = np.argmax(my_docs_topic[i,:])
    #print("Document " + str(i+1) + " = " + topic_names[topic_pick])
    labels.append(topic_names[topic_pick])

In [175]:
labels

['Music',
 'Music',
 'Social',
 'Book',
 'Music',
 'Social',
 'Music',
 'Social',
 'Social',
 'Music',
 'Social',
 'Movie',
 'Social',
 'Social',
 'Music',
 'Social',
 'Music',
 'Music',
 'Social',
 'Movie',
 'Movie',
 'Music',
 'Social',
 'Music',
 'Music',
 'Social',
 'Social',
 'Music',
 'Social',
 'Music',
 'Book',
 'Book',
 'Social',
 'Social',
 'Music',
 'Social',
 'Social',
 'Movie',
 'Book',
 'Book',
 'Music',
 'Music',
 'Music',
 'Book',
 'Book',
 'Music',
 'Social',
 'Music',
 'Music',
 'Social',
 'Social',
 'Social',
 'Music',
 'Movie',
 'Movie',
 'Social',
 'Music',
 'Music',
 'Social',
 'Social',
 'Movie',
 'Music',
 'Social',
 'Social',
 'Social',
 'Music',
 'Social',
 'Movie',
 'Social',
 'Social',
 'Book',
 'Music',
 'Movie',
 'Music',
 'Book',
 'Social',
 'Music',
 'Music',
 'Social',
 'Movie',
 'Social',
 'Music',
 'Social',
 'Book',
 'Social',
 'Book',
 'Social',
 'Music',
 'Movie',
 'Movie',
 'Social',
 'Social',
 'Social',
 'Music',
 'Music',
 'Social',
 'Music',
 

In [176]:
len(labels)

59946

In [177]:
#Nueva columna en pandas con las etiquetas
profiles['labels'] = labels

In [178]:
profiles.head(2)

,age,status,sex,orientation,body_type,diet,drinks,drugs,education,ethnicity,...,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,all_essays,labels
0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,"asian, white",...,making people laugh. ranting about a good salt...,"the way i look. i am a six foot half asian, ha...","books: absurdistan, the republic, of mice and ...",food. water. cell phone. shelter.,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,Music
1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,white,...,being silly. having ridiculous amonts of fun w...,,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories. my b...,,,i am very open and will share just about anyth...,,i am a chef: this is what that means. 1. i am ...,Music


In [181]:
etiquetado = profiles['labels']

In [182]:
etiquetado.to_csv("labels_essays.csv", index = True)

### Drop columnas essay0...essay9

In [ ]:
#Drop essay0...essay9 individuales
profiles.drop(labels=['essay0','essay1','essay2','essay3','essay4','essay5','essay6','essay7','essay8','essay9'], axis=1, inplace=True)